# Recomendador basado en contenidos
Consiste en recomendar contenido similar a un determinado contenido

#### Principales caracteristicas
- Sólo necesita la información de un producto, por lo que no se necesita información histórica
- Se suele usar para evitar problemas de arranque en frio

#### Como lo hacemos
1. Definir una métrica de similitud para comparar los contenidos
2. Calcular la similitud entre todos los elementos con la métrica seleccionada
3. (Opcional) Definir vecindarios para optimizar


## Antes de comenzar... veamos el dataset
Recursos turisticos de Andalucia obtenidos de [TripAdvisor](https://www.tripadvisor.es)

In [1]:
import pandas as pd

recursos = pd.read_csv('./dataset/Recursos_turisticos.csv', sep='|')
recursos.head(5)

,_id,nombre_ws,strCategoria,strSubcategoria,strCiudad,strDescripcion
0,5880b148636e564ad8d9b952,Hotel Albucasis,Alojamientos,Hoteles,Córdoba,.El hotel tiene a disposición de sus clientes ...
1,5880b148636e564ad8d9b963,Hotel Sierra Hidalga,Alojamientos,Hoteles,Ronda,Enclavado en una situación privilegiada con fa...
2,5880b1f2636e564ad8d9b969,Hotel Virgen de los Reyes,Alojamientos,Hoteles,Sevilla,"El Hotel Virgen de los Reyes***, ubicado en el..."
3,5880b1f2636e564ad8d9b96a,Hotel An Santo Domingo,Alojamientos,Hoteles,Lucena,El Hotel Santo Domingo Lucena está situado en ...
4,5880b1f2636e564ad8d9b96f,Leflet Valme,Alojamientos,Hoteles,Dos Hermanas,El Hotel Leflet Valme está situado estratégica...


## Métrica de similitud para texto

Una de las técnicas más utilizadas para obtener similitude texto es TF-IDF, tecnica proveniente del campo de Informatin Retrieval.

TF-IDF permite realizar una representación vectorial de los textos. Por ejemplo:

> "En un lugar de la mancha"

|_|En|un |lugar|de|la|mancha|
|--|--|---|-----|--|--|------|
|TF |1|1|1|1|1|1|
|DF |200|1700|100|3000|4075|3|
|IDF|1/200|1/1700|1/100|1/3000|1/4075|1/3|

Al tener una representación vectorial de los textos podemos usar medidas tradicionale
<img src="./imagenes/similitudes.png">

### Ejemplo TF-IDF

**Paso 1** Preparacion y tokenizado de datos

In [2]:
import gensim
import nltk
import tempfile
import heapq

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

documentos = recursos['strDescripcion']
print ("Numero de documentos:", len(documentos))
print ("\n", documentos[1])

#Tokenización
docs_tokenizado = [[w for w in word_tokenize(text)] 
             for text in documentos]
print ("\n\nElementos tokenizados\n")
print (docs_tokenizado[1])

# Convertimos a minúsculas
docs_tokenizado = [[w.lower() for w in documento] 
             for documento in docs_tokenizado]
print ("\n\nTodo minúsculas\n")
print (docs_tokenizado[1])

#Quitamos puntaciones
import string
table = str.maketrans('', '', string.punctuation)
docs_tokenizado = [[w.translate(table) for w in documento] 
             for documento in docs_tokenizado]
docs_tokenizado = [[w for w in documento if w.isalpha()] 
             for documento in docs_tokenizado]

print ("\n\nPuntuaciones\n")
print (docs_tokenizado[1])

#Quitamos StopWords
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
docs_tokenizado = [[w for w in documento if not w in stop_words] 
             for documento in docs_tokenizado]

print ("\n\nSin stop words\n")
print (docs_tokenizado[1])


C:\Users\Mario\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Numero de documentos: 601

 Enclavado en una situación privilegiada con fantásticas vistas a la sierra, y a menos de 2 Km. del centro de la ciudad de Ronda y 45 Km. de la Costa del Sol, es un lugar ideal para disfrutar de la oferta cultural y gastronómica además del patrimonio histórico-artístico de la ciudad, pudiendo combinarla con actividades en su entorno natural como la Sierra de las Nieves (reserva de la biosfera). Su decoración de estilo moderno y actual ofrece un ambiente selecto y acogedor. En nuestro restaurante podrán degustar los platos de nuestro cocinero Antonio Castro Aguilar además de una exquisita carta de vinos..


Elementos tokenizados

['Enclavado', 'en', 'una', 'situación', 'privilegiada', 'con', 'fantásticas', 'vistas', 'a', 'la', 'sierra', ',', 'y', 'a', 'menos', 'de', '2', 'Km', '.', 'del', 'centro', 'de', 'la', 'ciudad', 'de', 'Ronda', 'y', '45', 'Km', '.', 'de', 'la', 'Costa', 'del', 'Sol', ',', 'es', 'un', 'lugar', 'ideal', 'para', 'disfrutar', 'de', 'la', 'o

**Paso 2** Creación de un un diccionario con todo el vocabulario

In [3]:
diccionario = gensim.corpora.Dictionary(docs_tokenizado)

print("Número de palabras en el diccionario:",len(diccionario))
for i in range(len(diccionario)):
    print(i, diccionario[i])

Número de palabras en el diccionario: 7189
0 albolafia
1 amantes
2 antiguo
3 aves
4 barrio
5 belleza
6 bus
7 calle
8 cama
9 ciudad
10 clientes
11 coche
12 cuna
13 córdoba
14 describir
15 diferentes
16 disposición
17 encuentra
18 especies
19 guadalquivir
20 hermosa
21 hotel
22 información
23 judería
24 mezquita
25 minutos
26 monumentos
27 morena
28 museos
29 naturaleza
30 observar
31 oficina
32 pelo
33 pie
34 posee
35 puede
36 pueden
37 recomendamos
38 relajarnos
39 restaurantes
40 ribera
41 rincón
42 rio
43 secador
44 sierra
45 situados
46 sotos
47 supletoria
48 tabernas
49 tanta
50 televisión
51 tranquilo
52 turismo
53 turísticaestamos
54 típica
55 visitar
56 acogedor
57 actividades
58 actual
59 además
60 aguilar
61 ambiente
62 antonio
63 biosfera
64 carta
65 castro
66 centro
67 cocinero
68 combinarla
69 costa
70 cultural
71 decoración
72 degustar
73 disfrutar
74 enclavado
75 entorno
76 estilo
77 exquisita
78 fantásticas
79 gastronómica
80 históricoartístico
81 ideal
82 km
83 lugar
84

783 fortaleza
784 fuerzas
785 goza
786 inigualable
787 jardines
788 jerónimo
789 jornadas
790 leyenda
791 localización
792 monasterio
793 necesitan
794 palacios
795 perfecta
796 pies
797 plaza
798 preparadas
799 real
800 sabika
801 san
802 servicio
803 triunfo
804 turísticas
805 ubicación
806 ubicados
807 universitaria
808 abledo
809 abrir
810 amplios
811 andalucía
812 andújar
813 barcafetería
814 beatriz
815 beza
816 cambio
817 capital
818 cazorla
819 cercanía
820 ciudades
821 comida
822 comunican
823 corto
824 cuidadosamente
825 decoradas
826 enclave
827 estrellas
828 fácil
829 geográfico
830 gestión
831 inmejorable
832 jardín
833 jaénrecientemente
834 linares
835 lleves
836 madrid
837 mediterránea
838 mengíbar
839 nosotroscompleta
840 panorámica
841 permiten
842 principales
843 propiedad
844 puertas
845 recuerdo
846 santa
847 sevillael
848 silva
849 tiempo
850 tras
851 viajan
852 vuelto
853 vías
854 úbeda
855 alcázar
856 atención
857 atracciones
858 bodegas
859 caballos
860 calidadp

1654 exquisitos
1655 flamencos
1656 jeep
1657 llenos
1658 malvasia
1659 malvasía
1660 marismeña
1661 misma
1662 paseando
1663 paz
1664 permita
1665 pintoresca
1666 prefiera
1667 propicios
1668 refinado
1669 relajado
1670 restaurado
1671 rociera
1672 rosas
1673 salpicado
1674 sentirá
1675 sigue
1676 soñar
1677 sumérjase
1678 swing
1679 tradicional
1680 típicas
1681 viven
1682 vivir
1683 algeciras
1684 ciento
1685 cincuenta
1686 cuatro
1687 empleados
1688 mir
1689 motivo
1690 octavio
1691 recien
1692 reformadas
1693 simpatía
1694 situacion
1695 sorprender
1696 tamaño
1697 vamos
1698 vocación
1699 abades
1700 abanico
1701 adicionales
1702 amplitud
1703 carretera
1704 completa
1705 disfruten
1706 estacionamiento
1707 galería
1708 individuales
1709 máximo
1710 acogedoras
1711 acondicionadocalefacción
1712 acérquese
1713 aproveche
1714 artista
1715 bañarse
1716 caja
1717 canal
1718 coches
1719 consultar
1720 correo
1721 cortesía
1722 crea
1723 cálidas
1724 disponible
1725 electrónico
1726 eq

2599 chirimoyas
2600 conectan
2601 cristóbal
2602 cultivo
2603 frutas
2604 llamado
2605 mangos
2606 moriscos
2607 nerja
2608 orilla
2609 papayas
2610 primerísima
2611 separados
2612 subtropicales
2613 través
2614 adultos
2615 buceo
2616 calientesigualmente
2617 celebracionesen
2618 centralcaja
2619 centralcalefacción
2620 chapoteo
2621 cochescambio
2622 coffee
2623 colón
2624 cócteles
2625 deportivo
2626 directotv
2627 disponibilidad
2628 fuengirolauna
2629 hrslavanderíaalquiler
2630 jet
2631 minibarterrazaservicios
2632 monedaexcursionesgaraje
2633 náuticos
2634 octógono
2635 palmeras
2636 peloteléfono
2637 petición
2638 piano
2639 pingpong
2640 piscinahotel
2641 posibilidad
2642 reservas
2643 room
2644 rueda
2645 satéliteaire
2646 service
2647 sillas
2648 sky
2649 squash
2650 tiendaspeluqueríarestaurante
2651 vela
2652 corresponden
2653 profesional
2654 transformarse
2655 andalus
2656 arquitectónicas
2657 balcones
2658 centrales
2659 colección
2660 combinando
2661 cámara
2662 decorad

3347 límite
3348 proximidades
3349 sinagoga
3350 tendillas
3351 trinidad
3352 vivía
3353 abundan
3354 buffett
3355 carmelo
3356 ejecutivas
3357 moda
3358 mujeres
3359 peatonal
3360 remedios
3361 situa
3362 souvenirs
3363 triana
3364 ajardinadas
3365 conilfuerte
3366 constituido
3367 fontanilla
3368 gentes
3369 geográfica
3370 gestionado
3371 hospitalario
3372 llevar
3373 panorámicas
3374 pescadores
3375 responsable
3376 talante
3377 distingue
3378 golfel
3379 integración
3380 playamar
3381 sollocalizado
3382 tarik
3383 aficionados
3384 anualmente
3385 artificial
3386 cabinas
3387 celebra
3388 circuito
3389 ecuestres
3390 emplezamiento
3391 entrenamiento
3392 fifa
3393 gp
3394 greenfee
3395 hidroterapia
3396 homologadas
3397 icono
3398 ilimitado
3399 incentivo
3400 internacionales
3401 jack
3402 jogging
3403 montecastillo
3404 motociclismo
3405 nicklaus
3406 paddle
3407 polideportiva
3408 reglamentarias
3409 saunas
3410 sencillo
3411 sino
3412 turistas
3413 dominando
3414 enmarcado
3415

4253 cientos
4254 envuélvete
4255 evocadoras
4256 hechas
4257 mares
4258 misiana
4259 reservado
4260 adults
4261 air
4262 all
4263 bag
4264 beds
4265 beers
4266 body
4267 coffeetea
4268 completely
4269 complimentary
4270 conditioner
4271 conditioning
4272 dock
4273 double
4274 drinks
4275 dryer
4276 entire
4277 facilities
4278 facility
4279 formerly
4280 free
4281 gel
4282 hair
4283 hc
4284 heart
4285 inroom
4286 iphoneipod
4287 laundry
4288 led
4289 luggage
4290 machinefax
4291 makeup
4292 making
4293 max
4294 mirror
4295 of
4296 only
4297 per
4298 persons
4299 plazoleta
4300 printercopy
4301 rack
4302 reception
4303 redecorated
4304 renovated
4305 rental
4306 request
4307 rooms
4308 safe
4309 sat
4310 satellite
4311 shampoo
4312 single
4313 soap
4314 soft
4315 summer
4316 tassimo
4317 tea
4318 telephone
4319 types
4320 aguamar
4321 avanzada
4322 extraída
4323 innovadores
4324 occidental
4325 recomendado
4326 uspa
4327 archivo
4328 buganvillas
4329 colorido
4330 coquetos
4331 encantad

5120 definición
5121 dorm
5122 función
5123 futbolín
5124 fíocalor
5125 grabador
5126 hockey
5127 metrosnuevas
5128 neptuno
5129 nido
5130 niñosen
5131 personaspara
5132 serenatiene
5133 teens
5134 tostadora
5135 usb
5136 veranosalón
5137 acceda
5138 asiático
5139 galera
5140 gecko
5141 ibiza
5142 ocean
5143 texmex
5144 absoluta
5145 arboleda
5146 combinar
5147 depuración
5148 desarrollan
5149 esperar
5150 múltiples
5151 pradillo
5152 recreativo
5153 terracitas
5154 utensilios
5155 soleada
5156 burriana
5157 estupendos
5158 inexplorados
5159 paradisíacas
5160 piragua
5161 autopistas
5162 paddel
5163 clasificación
5164 merecida
5165 objetivo
5166 obtención
5167 obteniendose
5168 reinaugurado
5169 torreluz
5170 accesos
5171 asistencia
5172 disponibles
5173 mesasillas
5174 minimarket
5175 papeleras
5176 pensión
5177 públicos
5178 recibidor
5179 regímenes
5180 sanitaria
5181 socorristas
5182 tfno
5183 valoradas
5184 barsalón
5185 colgantes
5186 higuera
5187 panorámicos
5188 pisos
5189 play

5965 ventajosas
5966 acústico
5967 administrativa
5968 ajardinadatodas
5969 apta
5970 artería
5971 cadiz
5972 cerramiento
5973 ciudadtodas
5974 concedida
5975 consecutivamente
5976 correos
5977 datos
5978 desk
5979 drago
5980 full
5981 generales
5982 hd
5983 hospitality
5984 hotelhabitaciones
5985 metrostambién
5986 reconocida
5987 secadorinstalaciones
5988 servicesdisponemos
5989 tarda
5990 transmisión
5991 tvlcd
5992 ue
5993 urbanas
5994 viya
5995 wifinuestra
5996 últimos
5997 elviria
5998 roc
5999 apropiado
6000 embarque
6001 sintieran
6002 surgió
6003 tarifeñas
6004 tánger
6005 umbral
6006 vio
6007 adelantos
6008 etcestá
6009 forjada
6010 rl
6011 tecnológicos
6012 úbedacon
6013 com
6014 ecologico
6015 granja
6016 jardin
6017 opcion
6018 romantica
6019 brindando
6020 individualmente
6021 maroma
6022 orquídeas
6023 pantano
6024 romanes
6025 viñuela
6026 desear
6027 santana
6028 ahumado
6029 dotaciones
6030 exe
6031 factura
6032 palmera
6033 albaicin
6034 figura
6035 guardar
6036 guin

6804 trucha
6805 verduras
6806 aspecto
6807 barranquismo
6808 bóveda
6809 conforman
6810 fe
6811 isicio
6812 ladera
6813 paisajística
6814 plazoletas
6815 rafting
6816 rappel
6817 restringidas
6818 segundo
6819 trillo
6820 yedra
6821 adaptarse
6822 alza
6823 comenzó
6824 evolucionando
6825 fábrica
6826 harinas
6827 llenas
6828 reformarse
6829 abovedados
6830 darabenaz
6831 enlace
6832 marquesa
6833 multicolor
6834 cafeteríabar
6835 invernal
6836 ofrecerte
6837 wiffi
6838 escaparse
6839 montija
6840 rumor
6841 señalar
6842 diana
6843 discotecas
6844 distanciados
6845 oh
6846 adentrarse
6847 alicia
6848 basados
6849 cariñosamente
6850 carolina
6851 estratégico
6852 bordeado
6853 cibercorner
6854 vime
6855 quards
6856 està
6857 linia
6858 unico
6859 vik
6860 alegre
6861 allegro
6862 beiro
6863 campus
6864 colinda
6865 coronan
6866 sufrir
6867 terrazasolárium
6868 ubican
6869 adaptamos
6870 alcaidesa
6871 atraques
6872 concepción
6873 ohtels
6874 peñón
6875 sabrosa
6876 veracruz
6877 carga

**Paso 3** Generación del corpus de documentos, se calcula el número de veces que cada palabra aparece en el documento (TF)

In [4]:
corpus = [diccionario.doc2bow(gen_doc) for gen_doc in docs_tokenizado]
print(corpus[67])

[(21, 2), (66, 1), (96, 1), (162, 1), (164, 1), (287, 1), (311, 1), (573, 1), (683, 1), (1240, 1), (2054, 1), (2055, 1), (2056, 1), (2057, 1), (2058, 1), (2059, 1)]


**Paso 4** Generación del modelo para TF-IDF

In [5]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)
tf_idf[corpus[45]]

TfidfModel(num_docs=601, num_nnz=39167)
39167


[(9, 0.03818906765826256),
 (21, 0.007017519648339666),
 (44, 0.12833553278571688),
 (55, 0.08473806387007773),
 (81, 0.11309413793343841),
 (86, 0.06456064374157698),
 (96, 0.0904066592412939),
 (100, 0.06118525128123023),
 (116, 0.045641336075729434),
 (151, 0.12528442992852148),
 (183, 0.05303965194737251),
 (188, 0.13617475055844122),
 (196, 0.06377959427524632),
 (198, 0.07810027055817702),
 (208, 0.06702349777465344),
 (290, 0.038014339556300604),
 (296, 0.057170395569893305),
 (317, 0.39827942149950074),
 (392, 0.10720433948686617),
 (421, 0.06083101841742033),
 (477, 0.0870269193583654),
 (491, 0.09938779865872205),
 (498, 0.07870611968213573),
 (500, 0.10178978509753554),
 (520, 0.0819178524747632),
 (532, 0.06618158071405982),
 (546, 0.12833553278571688),
 (549, 0.1772957126616965),
 (567, 0.10720433948686617),
 (573, 0.09608881955656479),
 (578, 0.06048061517308904),
 (582, 0.1296706151036546),
 (583, 0.1772957126616965),
 (726, 0.09825041934773648),
 (738, 0.091242199324864

**Paso 5** Creación de un índice temporal

In [6]:
print (tempfile.gettempdir())
sims = gensim.similarities.Similarity(tempfile.gettempdir(),tf_idf[corpus],
                                      num_features=len(diccionario))
print (sims[tf_idf[corpus[45]]])

C:\Users\Mario\AppData\Local\Temp
[2.50748172e-02 4.67274413e-02 5.61061315e-03 1.50708761e-02
 4.91801463e-02 1.04745273e-02 2.41362154e-02 6.59954548e-02
 0.00000000e+00 2.94572264e-02 4.76162434e-02 1.40734226e-01
 6.46158308e-03 3.50262737e-03 2.03444567e-02 4.18958701e-02
 5.14933504e-02 2.20693485e-03 1.78291164e-02 2.28080709e-04
 4.02734727e-02 1.56663004e-02 4.14772741e-02 2.70768777e-02
 8.53183270e-02 1.04364552e-01 5.10980003e-02 8.83291513e-02
 3.23604280e-03 1.38040613e-02 4.02366593e-02 2.01346856e-02
 3.67798544e-02 0.00000000e+00 5.63181862e-02 2.58662701e-02
 3.72334570e-02 5.02345487e-02 3.63293141e-02 5.91653511e-02
 3.01821134e-03 1.89522840e-02 4.72832061e-02 4.33607809e-02
 1.80975571e-02 9.99999940e-01 1.93825066e-02 6.51541203e-02
 8.34397785e-03 1.39104128e-02 2.70864274e-02 1.81327143e-03
 5.38292155e-02 5.73998243e-02 3.42336074e-02 2.63522156e-02
 2.02940255e-02 6.90183043e-03 2.02929508e-02 1.19473413e-02
 6.09480310e-03 3.39296348e-02 4.56709713e-02 8.549

**Paso 6** Obtener elementos recomendados, los más similares

In [7]:
salida = sims[tf_idf[corpus[212]]]

heap = []
indice = 0
for medida in salida:
    heapq.heappush(heap, (medida, documentos[indice]))
    indice +=1

print (documentos[212])
heapq.nlargest(10,heap)

El tradicional diseño nazarí, la cúpula de azulejos, y las baldosas y arcos de estilo árabe encantan los huéspedes del Hotel Alhambra Palace. El hotel está situado a sólo 100 metros del monumento de La Alhambra, y está a 10-15 minutos a pie del centro de la ciudad de Granada. Las 126 habitaciones y suites ofrecen vistas impresionantes de la ciudad de Granada, Sierra Nevada y el bosque de la Alhambra, y muestran una bonita decoración tradicional y comodidades de lujo: climatización individual, conexión a Internet WIFI, TV de plasma y teléfono. Disfrute de una vista panorámica de la puesta de sol desde el restaurante o la terraza mientras saboreas nuestras delicias de gastronomía local.


[(0.9999999,
  'El tradicional diseño nazarí, la cúpula de azulejos, y las baldosas y arcos de estilo árabe encantan los huéspedes del Hotel Alhambra Palace. El hotel está situado a sólo 100 metros del monumento de La Alhambra, y está a 10-15 minutos a pie del centro de la ciudad de Granada. Las 126 habitaciones y suites ofrecen vistas impresionantes de la ciudad de Granada, Sierra Nevada y el bosque de la Alhambra, y muestran una bonita decoración tradicional y comodidades de lujo: climatización individual, conexión a Internet WIFI, TV de plasma y teléfono. Disfrute de una vista panorámica de la puesta de sol desde el restaurante o la terraza mientras saboreas nuestras delicias de gastronomía local.'),
 (0.22288907,
  'Este hotel en Granada de ambiente familiar está situado dentro de la ciudadela de la Alhambra. No solo es un Hotel cerca de la Alhambra, sino que es un Hotel en la Alhambra. Situación privilegiada que le permitirá disfrutar de la tranquilidad de la Alhambra estando cerc

## Nuevas técnicas, deep learning

Es una nueva técnica dentro del aprendizaje automático, basado en la estructura de las redes neuronales, pero usando arquitecturas profundas (deep) en la que múltiples capas son conectadas.

La principal característica y ventaja del deep learning es el **feature discovering** sin la necesidad de la intervención humana.

### Paragraph vector

- Algoritmo de aprendizaje no supervisado capaz de generar una representacion vectorial a partir de texto.
- Paragraph vector es capaz de aprender la semántica de las palabras.

<img src="./imagenes/doc2vec.jpg">

#### Ejemplo

**Paso 1** Entrenamiento del modelo

In [13]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Configuración
#-------------
__longitud_vector = 100
__ventana = 10
__frecuencia_minima = 5
__learning_rate = 0.025
__min_learning_rate = 0.025
__epoch = 100
#---------------------

                
tagged_data = [TaggedDocument(words=word_tokenize(_d .lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]
model = gensim.models.Doc2Vec(vector_size=__longitud_vector, window=__ventana, min_count=__frecuencia_minima, workers=11,alpha=__learning_rate, min_alpha=__min_learning_rate, epochs=__epoch)
model.build_vocab(tagged_data)
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

#Recorrido de todos los documentos con los que se ha entrenado
for i in range(20):
    print (model.dv[i].tolist())
    


[0.632260262966156, -2.452707290649414, 0.13153041899204254, -2.5868606567382812, 1.708008885383606, -0.3161862790584564, 2.417945146560669, 1.2001609802246094, -0.8420687317848206, -0.6755121350288391, 0.22969914972782135, 1.1909563541412354, -0.904654860496521, 0.637587308883667, 1.9750844240188599, -2.4613637924194336, 1.3739748001098633, 2.544607162475586, -1.4448306560516357, -3.7232067584991455, 1.1318316459655762, -1.4141345024108887, 1.82809579372406, -3.4401049613952637, 0.14412684738636017, -2.630280017852783, 0.13698293268680573, -1.286433458328247, 0.6815294027328491, 3.842562437057495, 2.4708352088928223, -1.2521681785583496, 1.058517336845398, 0.5908907651901245, -0.12311016768217087, -0.2224106639623642, -1.3829461336135864, 1.041250228881836, -0.26543089747428894, -0.9323352575302124, 1.585966944694519, -2.0102016925811768, -0.242201030254364, -0.6310510039329529, 2.0628106594085693, 0.39831554889678955, 0.584092915058136, 0.5090688467025757, 1.9963929653167725, -2.1161

**Paso 2** Cálculo de similitud

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

elemento =model.dv[100]

heap = []
for i in range(len(model.dv)):
    sim =cosine_similarity(elemento.reshape(1, -1), model.dv[i].reshape(1, -1))
    heapq.heappush(heap, (sim, documentos[i]))

print (documentos[100])
heapq.nlargest(10,heap)

Este bonito aparthotel se encuentra en la hermosa localidad de Zahara de los Atunes. Se trata de un complejo de apartamentos turísticos que ofrecen los servicios de un hotel. Ideal para un público con espí­ritu joven, ofrece estudios con literas, estudios con camas dobles, cunas, cocina equipada, microondas, baño privado con plato de ducha y secador, TV, DVD, calefacción, aire acondicionado y acceso a internet. Todo a muy buen precio, con un ambiente moderno y acogedor para que disfrute de la belleza de Cádiz y de las fabulosas playas de Zahara de los Atunes.


[(array([[1.]], dtype=float32),
  'Este bonito aparthotel se encuentra en la hermosa localidad de Zahara de los Atunes. Se trata de un complejo de apartamentos turísticos que ofrecen los servicios de un hotel. Ideal para un público con espí\xadritu joven, ofrece estudios con literas, estudios con camas dobles, cunas, cocina equipada, microondas, baño privado con plato de ducha y secador, TV, DVD, calefacción, aire acondicionado y acceso a internet. Todo a muy buen precio, con un ambiente moderno y acogedor para que disfrute de la belleza de Cádiz y de las fabulosas playas de Zahara de los Atunes.'),
 (array([[0.55424976]], dtype=float32),
  'El Playalinda Hotel está ubicado en Roquetas de Mar, a 50m de playa, junto al paseo marítimo peatonal y a 800 m del campo de golf Playa Serena. Ideal para quienes busquen un hotel acogedor, tranquilo y detallista en la costa de Almería.'),
 (array([[0.52756226]], dtype=float32),
  'El ATH Roquetas, un completo alojamiento de vacaciones en familia c

### Inferir vector para un nuevo texto
Una vez tenemos el modelo podemos inferir el vector para un nuevo texto, a continuación realizaremos un ejemplo

In [10]:
text = ["comer","marisco"]

#Obtener el texto de un nuevo vector
vector = model.infer_vector(text)
print(vector)

[-0.09728586 -0.03707353  0.0678668   0.19127043 -0.24596108 -0.06827786
 -0.1943415   0.06561632 -0.10167168 -0.2607504  -0.30034488 -0.09721062
 -0.0150125  -0.05464287  0.06428263 -0.4751373  -0.00875855 -0.04568084
 -0.04451621 -0.05895445  0.33617136  0.37246606  0.20248659  0.00503448
 -0.15629913  0.35858727 -0.08469708 -0.15006448 -0.08256242  0.315529
  0.28962412  0.16033322 -0.33428246 -0.43897125  0.29964244  0.4160386
  0.05270606  0.00112969 -0.34589353 -0.4641801   0.302669   -0.05967062
 -0.16523448 -0.13479868  0.22806606  0.05399456 -0.01311023 -0.11927786
  0.4673559   0.0330602   0.33928946 -0.15871286  0.04955858 -0.09053028
 -0.11436585  0.40228614  0.11222332  0.26332885 -0.02465895  0.0540069
 -0.29564616  0.23625702 -0.03168476 -0.2467347  -0.05149078  0.57791203
  0.18809411  0.0231405  -0.15902348 -0.00589976 -0.1639851   0.06928145
  0.34743288 -0.08631364 -0.02570864  0.15426709 -0.07055916 -0.03671778
 -0.5979578   0.06324041  0.18105349  0.11596754 -0.463

## ¿Podemos usar imágenes?
El uso de las nuevas técnicas de deep learning han provocado un gran avance en la visión artificial, una de las técnicas más usadas son las redes neuronales de convolución(CNN)

### CNN
<img src="./imagenes/cnn.png">

Extracción de *feature* de alto nivel, obteniendo los valores de la última capa antes de clasificación.

Algunos modelos de CNN:
* **AlexNet**: el primer modelo de CNN usado para el reto ILSVRC en 2012, mejoró más de un 10% los resultados anteriores.
* **GoogLenet** (2014):  aparecen bifucarciones en la arquitectura (inception) por primera vez.
* **ResNet**: arquitectura más "profunda", 152 capas.

### Ejemplo de extracción de features
<img src="./imagenes/Presentacion2.png">

In [11]:
featuresImagenes = pd.read_csv('./dataset/VectoresRopa.csv', sep='|', header=None)
featuresImagenes.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,Desigual,0,0.44,0.57,0.54,0.00,0.0,0.04,0.08,0.04,...,0.00,0.10,0.77,0.91,0.00,2.99,0.00,0.02,0.00,0.00
1,Desigual,1,0.16,0.02,0.00,0.28,0.0,0.19,0.14,0.13,...,0.00,0.11,0.00,0.90,0.54,0.88,4.57,0.56,1.13,0.00
2,Desigual,10,0.14,0.34,0.93,0.00,0.0,0.61,0.33,0.01,...,0.00,0.43,0.47,1.00,0.00,1.46,0.00,0.16,0.00,0.01
3,Desigual,100,0.00,0.12,1.12,0.15,0.0,0.00,0.00,0.00,...,0.34,1.85,0.00,0.38,0.25,0.00,0.00,0.00,0.00,0.00
4,Desigual,1000,0.00,0.03,0.10,0.00,0.0,0.03,0.00,0.08,...,0.00,0.01,0.83,0.29,0.28,0.05,0.00,1.16,2.02,0.00


### Cálculo de similitud

In [12]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

imagenRecomendar = 5
imagen = np.array(featuresImagenes.iloc[imagenRecomendar][2:])

heap = []
#for i in range(len(featuresImagenes)):
for i in range(100):
    if(i!=5):
        imagenComparar = np.array(featuresImagenes.iloc[i][2:])
        sim =cosine_similarity(imagen.reshape(1, -1), imagenComparar.reshape(1, -1))
        heapq.heappush(heap, (sim, featuresImagenes.iloc[i][0]+ '-'  + str(featuresImagenes.iloc[i][1])))

#print documentos[45]
heapq.nlargest(10,heap)

[(array([[0.7542513]]), 'Desigual-1045'),
 (array([[0.72954216]]), 'Desigual-1033'),
 (array([[0.71957988]]), 'Desigual-1036'),
 (array([[0.71644258]]), 'Desigual-1054'),
 (array([[0.7111745]]), 'Desigual-1031'),
 (array([[0.71089836]]), 'Desigual-1070'),
 (array([[0.69953612]]), 'Desigual-1047'),
 (array([[0.69655343]]), 'Desigual-1030'),
 (array([[0.69131285]]), 'Desigual-1032'),
 (array([[0.68986881]]), 'Desigual-1008')]

<img src="./imagenes/Presentacion1.png">